In [ ]:
!pip install langchain_community groq langgraph python-dotenv faiss-cpu pypdf PyPDFLoader

In [ ]:
!curl -fsSL https://raw.githubusercontent.com/arduino/arduino-cli/master/install.sh | sh
!arduino-cli version

In [12]:
import os

os.environ["PATH"] += os.pathsep + "/content/bin"
!arduino-cli version
!arduino-cli core update-index
!arduino-cli core install arduino:avr

arduino-cli  Version: 1.3.1 Commit: 08ff7e2b Date: 2025-08-28T13:51:45Z


In [23]:
import os
import subprocess

# Helper function to create a fresh sketch folder
def create_sketch_folder(sketch_name, code_str):
    folder_path = f"/content/{sketch_name}"
    os.makedirs(folder_path, exist_ok=True)
    file_path = os.path.join(folder_path, f"{sketch_name}.ino")
    with open(file_path, "w") as f:
        f.write(code_str)
    return folder_path

# Function to validate Arduino sketch
def validate_arduino_sketch(sketch_folder, board_fqbn="arduino:avr:uno"):
    result = subprocess.run(
        ["arduino-cli", "compile", "--fqbn", board_fqbn, sketch_folder],
        capture_output=True, text=True
    )
    if result.returncode == 0:
        return True, "Compilation successful ✅"
    else:
        return False, result.stderr

# Example: generate first sketch
code1 = """
void setup() {
  pinMode(13, OUTPUT);
}

void loop() {
  digitalWrite(13, HIGH);
  delay(1000);
  digitalWrite(13, LOW);
  delay(1000);
}
"""
sketch1_folder = create_sketch_folder("blink_sketch", code1)
valid, message = validate_arduino_sketch(sketch1_folder)
print(valid, message)

# Example: generate second sketch
code2 = """
void setup() {
  pinMode(12, OUTPUT);
}

void loop() {
  digitalWrite(12, HIGH);
  delay(500);
  digitalWrite(12, LOW);
  delay(500);
}
"""
sketch2_folder = create_sketch_folder("blink2_sketch", code2)
valid2, message2 = validate_arduino_sketch(sketch2_folder)
print(valid2, message2)


True Compilation successful ✅
True Compilation successful ✅


In [ ]:
import os
import ast
import re
import getpass
import shutil
import tempfile
import subprocess
from typing import List, TypedDict, Literal
from groq import Groq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, END
import subprocess
import sys

arduino_cli_path = shutil.which('arduino-cli')
if arduino_cli_path:
    print(f"Found 'arduino-cli' at: {arduino_cli_path}. Will use it for validation.")
else:
    print("WARNING: 'arduino-cli' not found in PATH. Falling back to basic C++ syntax checks for Arduino code.")


def get_groq_api_key():
    """Securely gets the Groq API key from environment variables or user input."""
    if "GROQ_API_KEY" in os.environ:
        print("Found GROQ_API_KEY in environment variables.")
        return os.environ["GROQ_API_KEY"]
    else:
        print("GROQ_API_KEY not found in environment variables.")
        return getpass.getpass("Please enter your Groq API key: ")

def get_tavily_api_key():
    """Securely gets the Tavily API key for web search."""
    if "TAVILY_API_KEY" in os.environ:
        print("Found TAVILY_API_KEY in environment variables.")
        return os.environ["TAVILY_API_KEY"]
    else:
        print("TAVILY_API_KEY not found.")
        return getpass.getpass("Please enter your Tavily API key for web search: ")

groq_api_key = get_groq_api_key()
tavily_api_key = get_tavily_api_key()

if not groq_api_key or not tavily_api_key:
    print("One or more API keys are missing. Exiting.")
    exit()

client = Groq(api_key=groq_api_key)
os.environ["TAVILY_API_KEY"] = tavily_api_key
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [6]:
# --- Document Loading from PDF ---
def load_documents_from_directory(path: str) -> List:
    """Loads PDF documents from the specified directory."""
    print(f"Loading PDF documents from '{path}'...")
    if not os.path.isdir(path):
        print(f"Error: Directory '{path}' not found.")
        print("Please create the 'knowledge_base' directory and add your PDF book(s).")
        exit()

    loader = DirectoryLoader(path, glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True, use_multithreading=True)
    documents = loader.load()

    if not documents:
        print(f"No PDF documents found in '{path}'. The agent will rely solely on web search.")
    else:
        print(f"Successfully loaded {len(documents)} document(s).")
    return documents

# --- State Definition ---
class AgentState(TypedDict):
    question: str
    documents: List[str]
    generation: str
    is_code: bool
    code_type: Literal["python", "cpp", "none"]
    validation_error: str
    retries: int
    final_answer: str

# --- Graph Nodes ---

def retrieve_documents(state: AgentState):
    """Node to retrieve relevant documents from the vector store."""
    print("\n--- 📄 Retrieving Documents from PDF Content ---")
    question = state["question"]
    if retriever:
        docs = retriever.invoke(question)
        doc_contents = [d.page_content for d in docs]
        print(f"Retrieved {len(doc_contents)} relevant chunks for the query.")
        return {"documents": doc_contents}
    return {"documents": []}

def web_search(state: AgentState):
    """Node to perform a web search to gather additional, up-to-date context."""
    print("--- 🌐 Performing Web Search ---")
    question = state["question"]
    documents = state["documents"]

    search_tool = TavilySearchResults(max_results=3)
    try:
        search_results = search_tool.invoke(question)
        # Format the search results into a readable string
        formatted_results = "\n\n".join([f"Source: {res['url']}\nContent: {res['content']}" for res in search_results])
        print(f"Web search found {len(search_results)} results.")
        # Append web results to the existing documents from the PDF
        documents.append("\n\n--- Web Search Results ---\n" + formatted_results)
    except Exception as e:
        print(f"Web search failed: {e}. Proceeding without web context.")

    return {"documents": documents}


def generate_code_or_doc(state: AgentState):
    """Node to generate a response using the Llama 3.1 LLM with combined context."""
    print("--- 🤖 Generating Response with Llama 3.1 ---")
    question, documents, validation_error, retries = state['question'], state['documents'], state.get('validation_error'), state.get('retries', 0)

    system_prompt = (
        "You are an expert programmer specializing in Arduino (C++) and Raspberry Pi (Python). "
        "Your task is to provide complete, runnable code or clear technical explanations based on the user's question and the provided context. "
        "The context is a combination of information from a local PDF book and real-time web search results. "
        "You must intelligently synthesize this combined context to form accurate and well-structured answers."
    )

    user_prompt_parts = [f"Combined Context (from PDF and Web):\n{''.join(documents)}"]
    if validation_error:
        print(f"--- ⚠️ Retrying (Attempt {retries}) with validation error ---")
        user_prompt_parts.append(f"The previous code attempt failed with the following compiler error: '{validation_error}'. Please analyze the error and the context again to provide a corrected, complete script.")

    user_prompt_parts.append(f"User's Question: {question}")
    user_prompt = "\n\n".join(user_prompt_parts)

    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        temperature=0.6,
        max_tokens=8000,
    )
    generation = completion.choices[0].message.content

    is_python = any(k in generation for k in ["def ", "import RPi.GPIO", "import smbus"])
    is_cpp = any(k in generation for k in ["void setup()", "void loop()", "#include <Arduino.h>"])
    code_type = "python" if is_python else "cpp" if is_cpp else "none"
    print(f"Generation complete. Identified as code: {is_python or is_cpp}. Type: {code_type}.")
    return {"generation": generation, "is_code": is_python or is_cpp, "code_type": code_type, "retries": retries + 1}

def clean_code(code_string: str, language: str) -> str:
    """Improved helper to extract code from markdown fences."""
    pattern = rf"```{language}\s*\n(.*?)\n```"
    match = re.search(pattern, code_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    return code_string.replace("```", "").strip()

def validate_python_code(state: AgentState):
    """More specific validation for Python code using AST."""
    print("--- 🐍 Validating Python Code ---")
    code = clean_code(state["generation"], "python")
    if not code:
        return {"validation_error": "No Python code found to validate."}
    try:
        ast.parse(code)
        print("Validation successful: Python code is syntactically correct.")
        return {"validation_error": None}
    except (SyntaxError, IndentationError) as e:
        error_msg = f"Python Syntax Error: {e}"
        print(f"Validation failed: {error_msg}")
        return {"validation_error": error_msg}

def validate_cpp_code(state: AgentState):
    """
    Validates C++/Arduino code by attempting to compile it with 'arduino-cli'.
    Falls back to basic syntax checks if the CLI is not available.
    """
    # If arduino-cli is not found, use the fallback method
    if not arduino_cli_path:
        return validate_cpp_fallback(state)

    print("--- 🔬 Validating C++/Arduino Code with arduino-cli ---")
    code = clean_code(state["generation"], "cpp")
    if not code:
        return {"validation_error": "No C++ code found to validate."}

    # Use a temporary directory to create the sketch for compilation
    with tempfile.TemporaryDirectory() as temp_dir:
        sketch_name = "temp_sketch"
        sketch_path = os.path.join(temp_dir, sketch_name)
        os.makedirs(sketch_path)
        sketch_file_path = os.path.join(sketch_path, f"{sketch_name}.ino")

        with open(sketch_file_path, "w") as f:
            f.write(code)

        # Build the command to compile the sketch
        command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:uno", sketch_path]

        # Execute the command
        result = subprocess.run(command, capture_output=True, text=True, check=False)

        if result.returncode == 0:
            print("Validation successful: Arduino sketch compiled successfully.")
            return {"validation_error": None}
        else:
            # Extract the meaningful error from stderr
            error_message = result.stderr.strip()
            print(f"Validation failed: Compilation error.\n{error_message}")
            return {"validation_error": error_message}

def validate_cpp_fallback(state: AgentState):
    """Fallback validation for C++/Arduino code if arduino-cli is not present."""
    print("--- 🔬 Validating C++/Arduino Code (Fallback Method) ---")
    code = clean_code(state["generation"], "cpp")
    if not code:
        return {"validation_error": "No C++ code found to validate."}
    errors = []
    if "void setup()" not in code: errors.append("Missing 'void setup()' function.")
    if "void loop()" not in code: errors.append("Missing 'void loop()' function.")
    if code.count('{') != code.count('}'): errors.append("Mismatched curly braces {}.")
    if code.count('(') != code.count(')'): errors.append("Mismatched parentheses ().")
    if errors:
        error_msg = " | ".join(errors)
        print(f"Validation failed: {error_msg}")
        return {"validation_error": error_msg}
    else:
        print("Validation successful: C++/Arduino code seems structurally plausible.")
        return {"validation_error": None}

def prepare_final_output(state: AgentState):
    print("--- ✅ Preparing Final Output ---")
    return {"final_answer": state["generation"]}

# --- Conditional Edges for Routing ---
def route_to_validation(state: AgentState):
    return state["code_type"] if state["is_code"] else "end"

def route_after_validation(state: AgentState):
    if state.get("validation_error") and state.get("retries", 0) < MAX_RETRIES:
        return "retry"
    return "end"

# --- Main Execution Block ---
if __name__ == "__main__":
    MAX_RETRIES = 2
    KNOWLEDGE_BASE_PATH = "knowledge_base"

    print("\n--- Setting up RAG System for PDF Knowledge Base ---")
    raw_documents = load_documents_from_directory(KNOWLEDGE_BASE_PATH)
    retriever = None
    if raw_documents:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
        docs = text_splitter.split_documents(raw_documents)
        print(f"PDF content split into {len(docs)} chunks.")
        print("Initializing embedding model (this may take a moment)...")
        embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
        vectorstore = FAISS.from_documents(documents=docs, embedding=embedding_model)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
        print("RAG system is ready.")

    # --- Build the Graph ---
    workflow = StateGraph(AgentState)
    workflow.add_node("retrieve", retrieve_documents)
    workflow.add_node("web_search", web_search)
    workflow.add_node("generate", generate_code_or_doc)
    workflow.add_node("validate_python", validate_python_code)
    workflow.add_node("validate_cpp", validate_cpp_code)
    workflow.add_node("prepare_output", prepare_final_output)

    workflow.set_entry_point("retrieve")
    workflow.add_edge("retrieve", "web_search")
    workflow.add_edge("web_search", "generate")
    workflow.add_conditional_edges("generate", route_to_validation, {"python": "validate_python", "cpp": "validate_cpp", "none": "prepare_output", "end": "prepare_output"})
    workflow.add_conditional_edges("validate_python", route_after_validation, {"retry": "generate", "end": "prepare_output"})
    workflow.add_conditional_edges("validate_cpp", route_after_validation, {"retry": "generate", "end": "prepare_output"})
    workflow.add_edge("prepare_output", END)
    app = workflow.compile()

    # --- Run Example Queries ---
    queries = [
        "Based on the book, what is the C++ code for reading an analog sensor with an Arduino?",
        "Find the most popular Python library for controlling an I2C LCD screen with a Raspberry Pi and write a script to display 'Hello World'."
    ]
    for i, q in enumerate(queries):
        print(f"\n{'='*50}\n🚀 Executing Query {i+1}: '{q}'\n{'='*50}")
        inputs = {"question": q}
        result = app.invoke(inputs)

        print("\n\n--- ✨ Final Answer ✨ ---")
        print(result.get('final_answer', 'No final answer was generated.'))
        print("--------------------------\n")




--- Setting up RAG System for PDF Knowledge Base ---
Loading PDF documents from 'knowledge_base'...


100%|██████████| 3/3 [00:21<00:00,  7.21s/it]


Successfully loaded 750 document(s).
PDF content split into 859 chunks.
Initializing embedding model (this may take a moment)...
RAG system is ready.

🚀 Executing Query 1: 'Based on the book, what is the C++ code for reading an analog sensor with an Arduino?'

--- 📄 Retrieving Documents from PDF Content ---
Retrieved 2 relevant chunks for the query.
--- 🌐 Performing Web Search ---


/tmp/ipython-input-3518378152.py:49: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=3)


Web search found 3 results.
--- 🤖 Generating Response with Llama 3.1 ---
Generation complete. Identified as code: True. Type: cpp.
--- 🔬 Validating C++/Arduino Code (Fallback Method) ---
Validation successful: C++/Arduino code seems structurally plausible.
--- ✅ Preparing Final Output ---


--- ✨ Final Answer ✨ ---
Based on the provided PDF book and web search results, here's a complete and runnable C++ code for reading an analog sensor with an Arduino:

```cpp
const int ldr = A0;  // Define the analog sensor pin
const int senseLimit = 0;  // Initialize senseLimit to 0
const int NUMREADINGS = 10;  // Number of readings to take and average
int readings[NUMREADINGS];   // Array to hold readings
int index = 0;               // Index for readings array
int total = 0;               // Running total of readings
int average = 0;             // Average of readings
int val = 0;                 // Store the value from analogRead
int LED1 = 13;               // Pin for the first LED
int LED2 = 12

In [7]:
# --- Document Loading from PDF ---
def load_documents_from_directory(path: str) -> List:
    """Loads PDF documents from the specified directory."""
    print(f"Loading PDF documents from '{path}'...")
    if not os.path.isdir(path):
        print(f"Error: Directory '{path}' not found.")
        print("Please create the 'knowledge_base' directory and add your PDF book(s).")
        exit()

    loader = DirectoryLoader(path, glob="**/*.pdf", loader_cls=PyPDFLoader, show_progress=True, use_multithreading=True)
    documents = loader.load()

    if not documents:
        print(f"No PDF documents found in '{path}'. The agent will rely solely on web search.")
    else:
        print(f"Successfully loaded {len(documents)} document(s).")
    return documents

# --- State Definition ---
class AgentState(TypedDict):
    question: str
    documents: List[str]
    generation: str
    is_code: bool
    code_type: Literal["python", "cpp", "none"]
    validation_error: str
    retries: int
    final_answer: str

# --- Graph Nodes ---

def retrieve_documents(state: AgentState):
    """Node to retrieve relevant documents from the vector store."""
    print("\n--- 📄 Retrieving Documents from PDF Content ---")
    question = state["question"]
    if retriever:
        docs = retriever.invoke(question)
        doc_contents = [d.page_content for d in docs]
        print(f"Retrieved {len(doc_contents)} relevant chunks for the query.")
        return {"documents": doc_contents}
    return {"documents": []}

def web_search(state: AgentState):
    """Node to perform a web search to gather additional, up-to-date context."""
    print("--- 🌐 Performing Web Search ---")
    question = state["question"]
    documents = state["documents"]

    search_tool = TavilySearchResults(max_results=3)
    try:
        search_results = search_tool.invoke(question)
        # Format the search results into a readable string
        formatted_results = "\n\n".join([f"Source: {res['url']}\nContent: {res['content']}" for res in search_results])
        print(f"Web search found {len(search_results)} results.")
        # Append web results to the existing documents from the PDF
        documents.append("\n\n--- Web Search Results ---\n" + formatted_results)
    except Exception as e:
        print(f"Web search failed: {e}. Proceeding without web context.")

    return {"documents": documents}


def generate_code_or_doc(state: AgentState):
    """Node to generate a response using the Llama 3.1 LLM with combined context."""
    print("--- 🤖 Generating Response with Llama 3.1 ---")
    question, documents, validation_error, retries = state['question'], state['documents'], state.get('validation_error'), state.get('retries', 0)

    system_prompt = (
        "You are an expert programmer specializing in Arduino (C++) and Raspberry Pi (Python). "
        "Your task is to provide complete, runnable code or clear technical explanations based on the user's question and the provided context. "
        "The context is a combination of information from a local PDF book and real-time web search results. "
        "You must intelligently synthesize this combined context to form accurate and well-structured answers."
    )

    user_prompt_parts = [f"Combined Context (from PDF and Web):\n{''.join(documents)}"]
    if validation_error:
        print(f"--- ⚠️ Retrying (Attempt {retries}) with validation error ---")
        # Using a more generic "validation error" to be accurate for both compiler and fallback checks
        user_prompt_parts.append(f"The previous code attempt failed with the following validation error: '{validation_error}'. Please analyze the error and the context again to provide a corrected, complete script.")

    user_prompt_parts.append(f"User's Question: {question}")
    user_prompt = "\n\n".join(user_prompt_parts)

    completion = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
        temperature=0.6,
        max_tokens=8000,
    )
    generation = completion.choices[0].message.content

    is_python = any(k in generation for k in ["def ", "import RPi.GPIO", "import smbus"])
    is_cpp = any(k in generation for k in ["void setup()", "void loop()", "#include <Arduino.h>"])
    code_type = "python" if is_python else "cpp" if is_cpp else "none"
    print(f"Generation complete. Identified as code: {is_python or is_cpp}. Type: {code_type}.")
    return {"generation": generation, "is_code": is_python or is_cpp, "code_type": code_type, "retries": retries + 1}

def clean_code(code_string: str, language: str) -> str:
    """Improved helper to extract code from markdown fences."""
    pattern = rf"```{language}\s*\n(.*?)\n```"
    match = re.search(pattern, code_string, re.DOTALL)
    if match:
        return match.group(1).strip()
    return code_string.replace("```", "").strip()

def validate_python_code(state: AgentState):
    """More specific validation for Python code using AST."""
    print("--- 🐍 Validating Python Code ---")
    code = clean_code(state["generation"], "python")
    if not code:
        return {"validation_error": "No Python code found to validate."}
    try:
        ast.parse(code)
        print("Validation successful: Python code is syntactically correct.")
        return {"validation_error": None}
    except (SyntaxError, IndentationError) as e:
        error_msg = f"Python Syntax Error: {e}"
        print(f"Validation failed: {error_msg}")
        return {"validation_error": error_msg}

def validate_cpp_code(state: AgentState):
    """
    Validates C++/Arduino code by attempting to compile it with 'arduino-cli'.
    Falls back to basic syntax checks if the CLI is not available.
    """
    # If arduino-cli is not found, use the fallback method
    if not arduino_cli_path:
        return validate_cpp_fallback(state)

    print("--- 🔬 Validating C++/Arduino Code with arduino-cli ---")
    code = clean_code(state["generation"], "cpp")
    if not code:
        return {"validation_error": "No C++ code found to validate."}

    # Use a temporary directory to create the sketch for compilation
    with tempfile.TemporaryDirectory() as temp_dir:
        sketch_name = "temp_sketch"
        sketch_path = os.path.join(temp_dir, sketch_name)
        os.makedirs(sketch_path)
        sketch_file_path = os.path.join(sketch_path, f"{sketch_name}.ino")

        with open(sketch_file_path, "w") as f:
            f.write(code)

        # Build the command to compile the sketch
        command = [arduino_cli_path, "compile", "--fqbn", "arduino:avr:uno", sketch_path]

        # Execute the command
        result = subprocess.run(command, capture_output=True, text=True, check=False)

        if result.returncode == 0:
            print("Validation successful: Arduino sketch compiled successfully.")
            return {"validation_error": None}
        else:
            # Extract the meaningful error from stderr
            error_message = result.stderr.strip()
            print(f"Validation failed: Compilation error.\n{error_message}")
            return {"validation_error": error_message}

def validate_cpp_fallback(state: AgentState):
    """Fallback validation for C++/Arduino code if arduino-cli is not present."""
    print("--- 🔬 Validating C++/Arduino Code (Fallback Method) ---")
    code = clean_code(state["generation"], "cpp")
    if not code:
        return {"validation_error": "No C++ code found to validate."}
    errors = []
    if "void setup()" not in code: errors.append("Missing 'void setup()' function.")
    if "void loop()" not in code: errors.append("Missing 'void loop()' function.")
    if code.count('{') != code.count('}'): errors.append("Mismatched curly braces {}.")
    if code.count('(') != code.count(')'): errors.append("Mismatched parentheses ().")
    if errors:
        error_msg = " | ".join(errors)
        print(f"Validation failed: {error_msg}")
        return {"validation_error": error_msg}
    else:
        print("Validation successful: C++/Arduino code seems structurally plausible.")
        return {"validation_error": None}

def prepare_final_output(state: AgentState):
    print("--- ✅ Preparing Final Output ---")
    return {"final_answer": state["generation"]}

# --- Conditional Edges for Routing ---
def route_to_validation(state: AgentState):
    return state["code_type"] if state["is_code"] else "end"

def route_after_validation(state: AgentState):
    if state.get("validation_error") and state.get("retries", 0) < MAX_RETRIES:
        return "retry"
    return "end"

# --- Main Execution Block ---
if __name__ == "__main__":
    MAX_RETRIES = 2
    KNOWLEDGE_BASE_PATH = "knowledge_base"

    print("\n--- Setting up RAG System for PDF Knowledge Base ---")
    raw_documents = load_documents_from_directory(KNOWLEDGE_BASE_PATH)
    retriever = None
    if raw_documents:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
        docs = text_splitter.split_documents(raw_documents)
        print(f"PDF content split into {len(docs)} chunks.")
        print("Initializing embedding model (this may take a moment)...")
        embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")
        vectorstore = FAISS.from_documents(documents=docs, embedding=embedding_model)
        retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
        print("RAG system is ready.")

    # --- Build the Graph ---
    workflow = StateGraph(AgentState)
    workflow.add_node("retrieve", retrieve_documents)
    workflow.add_node("web_search", web_search)
    workflow.add_node("generate", generate_code_or_doc)
    workflow.add_node("validate_python", validate_python_code)
    workflow.add_node("validate_cpp", validate_cpp_code)
    workflow.add_node("prepare_output", prepare_final_output)

    workflow.set_entry_point("retrieve")
    workflow.add_edge("retrieve", "web_search")
    workflow.add_edge("web_search", "generate")
    workflow.add_conditional_edges("generate", route_to_validation, {"python": "validate_python", "cpp": "validate_cpp", "none": "prepare_output", "end": "prepare_output"})
    workflow.add_conditional_edges("validate_python", route_after_validation, {"retry": "generate", "end": "prepare_output"})
    workflow.add_conditional_edges("validate_cpp", route_after_validation, {"retry": "generate", "end": "prepare_output"})
    workflow.add_edge("prepare_output", END)
    app = workflow.compile()

    # --- Run Example Queries ---
    queries = [
        "Based on the book, what is the C++ code for reading an analog sensor with an Arduino?",
        "Find the most popular Python library for controlling an I2C LCD screen with a Raspberry Pi and write a script to display 'Hello World'."
    ]
    for i, q in enumerate(queries):
        print(f"\n{'='*50}\n🚀 Executing Query {i+1}: '{q}'\n{'='*50}")
        inputs = {"question": q}
        result = app.invoke(inputs)

        print("\n\n--- ✨ Final Answer ✨ ---")
        print(result.get('final_answer', 'No final answer was generated.'))
        print("--------------------------\n")




--- Setting up RAG System for PDF Knowledge Base ---
Loading PDF documents from 'knowledge_base'...


100%|██████████| 3/3 [00:22<00:00,  7.48s/it]


Successfully loaded 750 document(s).
PDF content split into 859 chunks.
Initializing embedding model (this may take a moment)...
RAG system is ready.

🚀 Executing Query 1: 'Based on the book, what is the C++ code for reading an analog sensor with an Arduino?'

--- 📄 Retrieving Documents from PDF Content ---
Retrieved 2 relevant chunks for the query.
--- 🌐 Performing Web Search ---
Web search found 3 results.
--- 🤖 Generating Response with Llama 3.1 ---
Generation complete. Identified as code: True. Type: cpp.
--- 🔬 Validating C++/Arduino Code (Fallback Method) ---
Validation successful: C++/Arduino code seems structurally plausible.
--- ✅ Preparing Final Output ---


--- ✨ Final Answer ✨ ---
Based on the book and the web search results, the C++ code for reading an analog sensor with an Arduino can be written as follows:

```cpp
const int ldr = A0;  // Analog sensor pin connected to A0

void setup() {
  pinMode(ldr, INPUT);  // Initialize the sensor as an input
  Serial.begin(9600);  //

In [10]:
try:
    image_bytes = app.get_graph().draw_mermaid_png()
    with open("workflow_graph.png", "wb") as f:
        f.write(image_bytes)
    print("\n--- 📊 Workflow graph image saved to workflow_graph.png ---")
except Exception as e:
    print(f"\n--- ⚠️ Could not generate graph image: {e} ---")
    print("Please ensure graphviz is installed and in your PATH ([https://graphviz.org/download/](https://graphviz.org/download/)).")


--- 📊 Workflow graph image saved to workflow_graph.png ---
